This file instantiates objects of the class Particle in the particle.ipynb file, using initial velocities and initial positions generated in the sampling.ipynb file.

Author: 18BME2104 Kaushal

In [1]:
from inspect import getmembers, isfunction, isclass # To inspect imported modules
from IPython.display import display, Math, Latex, clear_output # To use latex typesetting style

import numpy as np # For computations
import sympy as sm # For symbolic use

import pandas as pd # To use large tables
import matplotlib.pyplot as plt #To make plots
import csv # To read csv file of position and velocities required to initialize particles

In [ ]:
# Probably good idea to make plots, write to csv and other things in another notebook. Let's see.

In [3]:
class Step:
    def __init__(self, file_with_filenames = \
                 '/home/kushik/Kushik/VIT/Eighth semester/MagneticMirror/csvfiles/available files.csv'):
        self.file_with_filenames = file_with_filenames
        self.filenames = self.read_available_filenames(self.file_with_filenames)
        '''self.filenames would then be a list of availble files to read from'''
        
    def __string__(self):
        pass
        
    def docstring(self):
            pass
            # print (something)
            
    ### FILE READING SECTION
        
    def read_available_filenames(self):
        rows = []
        filenames = []
        with open(self.file_with_filenames, 'r') as csvfile:
            # creating a csv reader object
            csvreader = csv.reader(csvfile)

            # extracting field names through first row
            #fields = next(csvreader)

            # extracting each data row one by one
            for row in csvreader:
                rows.append(row)
        for row in rows:
            empty = ''
            for char in row:
                empty += char
            filenames.append(empty)
        return filenames
    
    def print_available_files(self):
        print(self.filenames)
    
    def read_r_or_v_file(self, index):
        '''
        index is the index on the self.filenames list of which the corresponding file is to be read.
        '''
        rows = []
        with open(self.filenames[index], 'r') as csvfile:
            csvreader = csv.reader(csvfile)

            for row in csvreader:
                rows.append(row)
        return row
    
    def reshaper(self, array_from_file):
        three_n = len(array_from_file)
        n = int(three_n / 3)
        reshaped_array = np.array(array_from_file).reshape((n, 3))
        return reshaped_array
        
    def read_r_or_v_file_and_reshape(self):
        array_from_file = self.read_r_or_v_file(index)
        reshaped_array = self.reshaper(array_from_file)
    
    ### PARTICELS INTIALIZATION SECTION
    
    
    ### FIELDS INITIALIZATION SECTION
    
    
    ### TIME STEP SECTION
    

In [ ]:
#create many particles
# by sampling different positions and velocities using various distribution functions
#Loop
    #get fields at their position
    #move the particles
    
#create new set of particles after some time, to model flow of particles/ionized gas 
#in a magnetron sputtering chamber

#For old particles that move out of the trap bounds (position exceeding some limits), or 
#modelingf particles being absorbed in the sputtered coating
#with an associated probability of absorption
#just delete those particles from the collection
#(but can be saved in csv files, so that they can stay recorded to be reused later)

In [1]:
print([1,2,3])

[1, 2, 3]
